# 과정1
 - 배경이 있는 영수증 사진의 4개의 꼭짓점을 지정해주면 영수증 부분만 scan해주는 작업

## Upload Image
 - 불러온 이미지를 축소하여 사용
 - 불러온 이미지를 그대로 사용함

In [10]:
# 불러온 이미지를 축소하여 사용
#pip install scikit-image
import numpy as np
import cv2, os
import matplotlib.pyplot as plt

from skimage.filters import threshold_local
from PIL import Image

# Sample file out of the dataset
file_name='D:\OCR/KakaoTalk_20211013_212329138.jpg'
filename, ext=os.path.splitext(os.path.basename(file_name))
img=Image.open(file_name)
#썸네일(thumbnail) 세로와 가로의 사이즈를 동일한 비율로 수정하여 썸네일을 만들수있게 하는 함수
#ANTIALIAS : 높은 해상도의 사진을 낮은 해상도로 변환할 때 깨지는 것을 방지
img.thumbnail((800, 800), Image.ANTIALIAS)
ori_img=np.array(img)

In [6]:
# 불러온 이미지를 그대로 사용하는 방식
import cv2, os
import numpy as np

img_path='D:\OCR/KakaoTalk_20211013_212329138.jpg'
filename, ext=os.path.splitext(os.path.basename(img_path))
ori_img=cv2.imread(img_path)

## Perspective Transform

In [11]:
src=[] #왼쪽위, 오른쪽위, 오른쪽아래, 왼쪽아래 점을 넣어놓을 전역변수

# mouse callback handler
def mouse_handler(event, x, y, flags, param):
  if event == cv2.EVENT_LBUTTONUP: #마우스를 눌렀다가 떼었을 때 아래를 시행
    img = ori_img.copy()

    src.append([x, y]) #위치를 src에 넣어줌

    for xx, yy in src:
      cv2.circle(img, center=(xx, yy), radius=5, color=(0, 255, 0), thickness=-1, lineType=cv2.LINE_AA) #클릭한 위치에 초록색 점을 그려줌

    cv2.imshow('img', img) #점을 찍은 이미지를 보여줌

    # perspective transform
    if len(src) == 4:
      src_np = np.array(src, dtype=np.float32) #src에 있는 변수를 np.float32로 변환

      #만들어낸 이미지의 가로길이=max(위쪽길이, 아래쪽길이)
      width = max(np.linalg.norm(src_np[0] - src_np[1]), np.linalg.norm(src_np[2] - src_np[3])) 
      #만들어낸 이미지의 세로길이=max(왼쪽길이, 오른쪽길이)  
      height = max(np.linalg.norm(src_np[0] - src_np[3]), np.linalg.norm(src_np[1] - src_np[2]))

      dst_np = np.array([
        [0, 0],
        [width, 0],
        [width, height],
        [0, height]
      ], dtype=np.float32)

      M = cv2.getPerspectiveTransform(src=src_np, dst=dst_np) #M : 어떻게 변환해야할지 알려주는 행렬
      result = cv2.warpPerspective(ori_img, M=M, dsize=(width, height))

      cv2.imshow('result', result)
      cv2.imwrite('D:\OCR\pre_data/%s_result%s' % (filename, ext), result) #완성된 이미지를 파일명_result를 붙여서 저장한다

In [12]:
# main
cv2.namedWindow('img') #window 이름 지정
cv2.setMouseCallback('img', mouse_handler) 

cv2.imshow('img', ori_img)
cv2.waitKey(0)

<ipython-input-11-fef9a78b8ca6>:32: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  result = cv2.warpPerspective(ori_img, M=M, dsize=(width, height))


-1